In [1]:
import dismech
import sympy as sp

In [2]:
import sympy as sp
from sympy.tensor.array import derive_by_array

# Utility functions
def dot_product(x0, x1, x2, y0, y1, y2):
    return x0 * y0 + x1 * y1 + x2 * y2

def norm(x0, x1, x2):
    return sp.sqrt(x0**2 + x1**2 + x2**2)

def smooth_abs(x, epsilon=1e-6):
    return sp.sqrt(x**2 + epsilon)

def cross(x0, x1, x2, y0, y1, y2):
    return (
        x1 * y2 - x2 * y1,
        x2 * y0 - x0 * y2,
        x0 * y1 - x1 * y0,
    )

def piecewise_abs(x):
    return sp.Piecewise(
        (x, x > 0),
        (-x, x < 0),
        (0, True)  # At x = 0
    )

# Delta functions
def delta_p_to_p(x0, x1, x2, y0, y1, y2, a0, a1, a2, b0, b1, b2):
    return norm(a0 - x0, a1 - x1, a2 - x2)

def delta_p_to_e(x0, x1, x2, y0, y1, y2, a0, a1, a2, b0, b1, b2):
    u0, u1, u2 = cross(y0 - x0, y1 - x1, y2 - x2, y0 - a0, y1 - a1, y2 - a2)
    return norm(u0, u1, u2) / norm(y0 - x0, y1 - x1, y2 - x2)

def delta_e_to_e(x0, x1, x2, y0, y1, y2, a0, a1, a2, b0, b1, b2):
    u0, u1, u2 = cross(y0 - x0, y1 - x1, y2 - x2, b0 - a0, b1 - a1, b2 - a2)
    u_norm = norm(u0, u1, u2)
    proj = dot_product(x0 - a0, x1 - a1, x2 - a2, u0 / u_norm, u1 / u_norm, u2 / u_norm)
    return piecewise_abs(proj)

# Define all 12 variables: x, y, a, b
vars = sp.symbols('x0 x1 x2 y0 y1 y2 a0 a1 a2 b0 b1 b2', real=True)
x0, x1, x2, y0, y1, y2, a0, a1, a2, b0, b1, b2 = vars

# Wrap delta functions
delta_pp = delta_p_to_p(*vars).doit()
delta_pe = delta_p_to_e(*vars).doit()
delta_ee = delta_e_to_e(*vars).doit()

# Compute gradient and Hessian for each
grad_pp = derive_by_array(delta_pp, vars).doit()
hess_pp = derive_by_array(grad_pp, vars).doit()

grad_pe = derive_by_array(delta_pe, vars).doit()
hess_pe = derive_by_array(grad_pe, vars).doit()

grad_ee = derive_by_array(delta_ee, vars).doit()
hess_ee = derive_by_array(grad_ee, vars).doit()

# Example: print gradient and Hessian shapes
print("delta_p_to_p:")
print("  Gradient:", grad_pp)
print("  Hessian:", hess_pp)

print("\ndelta_p_to_e:")
print("  Gradient:", grad_pe)
print("  Hessian:", hess_pe)

print("\ndelta_e_to_e:")
print("  Gradient:", grad_ee)
print("  Hessian:", hess_ee)


delta_p_to_p:
  Gradient: [(-a0 + x0)/sqrt((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2), (-a1 + x1)/sqrt((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2), (-a2 + x2)/sqrt((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2), 0, 0, 0, (a0 - x0)/sqrt((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2), (a1 - x1)/sqrt((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2), (a2 - x2)/sqrt((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2), 0, 0, 0]
  Hessian: [[(-a0 + x0)*(a0 - x0)/((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2)**(3/2) + 1/sqrt((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2), (a0 - x0)*(-a1 + x1)/((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2)**(3/2), (a0 - x0)*(-a2 + x2)/((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2)**(3/2), 0, 0, 0, (a0 - x0)**2/((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2)**(3/2) - 1/sqrt((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2), (a0 - x0)*(a1 - x1)/((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2)**(3/2), (a0 - x0)*(a2 - x2)/((a0 - x0)**2 + (a1 - x1)**2 + (a2 - x2)**2)**(3/2), 0, 0, 0], [(-a0 + x0)*(a1 - x1

In [3]:
pp_func = sp.lambdify(vars, delta_pp, modules='numpy')
grad_pp_func = sp.lambdify(vars, grad_pp, modules='numpy')
hess_pp_func = sp.lambdify(vars, hess_pp, modules='numpy') 

ee_func = sp.lambdify(vars, delta_ee, modules='numpy')
grad_ee_func = sp.lambdify(vars, grad_ee, modules='numpy')
hess_ee_func = sp.lambdify(vars, hess_ee, modules='numpy') 

In [14]:
import numpy as np

# Make up some test inputs for the 12 variables (x0, x1, x2, y0, ..., b2)
test_input = np.array([
    0.0, 1.0, 0.0,  # x
    0.0, 1.0, 0.0,  # y
    0.0, 1.01, 0.0,  # a
    0.0, 2.01, 0.0   # b
])

"""
test_input = np.array([
    0.0, 0.0, 0.2,  # x
    1.0, 1.0, 0.0,  # y
    0.0, 1.0, 0.5,  # a
    1.0, 0.0, 0.0   # b
])

"""


'\ntest_input = np.array([\n    0.0, 0.0, 0.2,  # x\n    1.0, 1.0, 0.0,  # y\n    0.0, 1.0, 0.5,  # a\n    1.0, 0.0, 0.0   # b\n])\n\n'

In [12]:
def get_E(Delta, delta, h, k_1):
    return sp.Piecewise(
        ((2 * h - Delta) ** 2, sp.And(Delta > 0, Delta <= 2 * h - delta)),
        ((1 / k_1 * (sp.log(1 + sp.exp(k_1 * (2 * h - Delta)))))**2, sp.And(Delta > 2 * h - delta, Delta < 2 * h + delta)),
        (0, True),
    )

Delta = sp.symbols('Delta', real=True)
delta = 0.2
h = 0.02
k_1 = 15 / delta
E_func = sp.lambdify(Delta, get_E(Delta, delta, h, k_1), modules='numpy')
grad_E_func = sp.lambdify(Delta, sp.diff(get_E(Delta, delta, h, k_1), Delta), modules='numpy')
hess_E_func = sp.lambdify(Delta, sp.diff(get_E(Delta, delta, h, k_1), Delta, Delta), modules='numpy')

In [15]:
print(k_1)
def grad_E_ee(x):
    d = pp_func(*x)
    print(x)
    print(d)
    print(d < 2 * h + delta and d > 2 * h - delta)
    return grad_pp_func(*x) * grad_E_func(d)

grad_E_ee(test_input)

75.0
[0.   1.   0.   0.   1.   0.   0.   1.01 0.   0.   2.01 0.  ]
0.010000000000000009
True


array([-0.        ,  0.05669642, -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.05669642, -0.        , -0.        ,
       -0.        , -0.        ])

test_input = np.array([0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.01, 0.0, 0.0, 2.01, 0.0 ,0.00010000000000000018,0.02,75])